Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menu bar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menu bar, select Cell$\rightarrow$Run All).

Below, please fill in your name and collaborators:

In [1]:
NAME = "Jeffrey Keomany"
COLLABORATORS = ""

# Assignment 3 - Time Series Analysis
**(15 points total)**

## Assignment tasks:

In this assignment you will conduct time series analysis of the financial data. 

1. Setup your environment to access and download latest stock data. Please see instructions below for different tools you can use to get the data. You can use any of the options provided, either Quandl or Yahoo Finance. If you know of any other service to download the data, please use that service, provide an explanation in the comments.

2. *(2 points)* Download the **adjusted** close prices for FB, MMM, IBM and AMZN for the last 60 months. If you run into any issues downloading the data from online sources, you can use `.csv` files provided. This will not affect your grade for the assignment.

3. *(3 points)* Resample the data to get prices for the end of the **business** month. Select the **Adjusted Close** for each stock.

4. *(3 points)* Use the pandas `autocorrelation_plot()` function to plot the autocorrelation of the adjusted month-end close prices for each of the stocks.
    - Are they autocorrelated?
    - Provide short explanation.

5. *(4 points)* 
    - Calculate the monthly returns for each stock using the "shift trick" explained in the lecture, using `shift()` function. 
    - Use pandas `autotocorrelation_plot()` to plot the autocorrelation of the monthly returns.
    - Are the returns autocorrelated? Provide short explanation.

6. *(3 points)*
    - Combine all 4 time series (returns) into a single DataFrame,
    - Visualize the correlation between the returns of all pairs of stocks using a scatter plot matrix (use `scatter_matrix()` function from `pandas.plotting`).
    - Explain the results. Is there any correlation?

**NOTES:** 
1. In this assignment, please make sure the DataFrame(s) do not contain any NAs before you plot autocorrelations or scatter matrix.
2. Both options explained below use `pandas-datareader` package for remote data access. To install it, type the following in a command window: `conda install pandas-datareader`. You will also need to install one or more of the following packages `fix_yahoo_finance`, `quandl`. See below.

---------

## Downloading Stock Prices

### Option 1 - Using QUANDL

To use QUANDL service, you need to create an account and get an API Key. Here is the short description of steps:

- Go to https://www.quandl.com/
- Click either `sign up` at the top right corner of the home page, or scroll all the way down and click `Create Free Account` button at the bottom of the page.
- Create an account. 
- You will receive an email to the email address you have used during the registration. Confirm your email.

You are all set.

Now, as you login into your account, click the avatar icon at the top right corner of the page, select `"Account Settings."`
On the next page, you will see `Your API Key` field with a long string of numbers and characters underneath. You need this API key for your call to Quandl from the notebook. In the code below, replace `YOUR_API_KEY` with the actual API key from your account. 

**NOTE**: You can remove this key before submitting the assignment.

In [2]:
# all imports and env variables
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
import datetime
import pandas_datareader.data as web

# This line of code should work on Windows and Mac
#%env QUANDL_API_KEY = "YOUR_API_KEY"

# If the above line of code does not work on your system,
# You can use this way of setting Quandl env variable
import quandl
quandl.ApiConfig.api_key = "YOUR_API_KEY"

ModuleNotFoundError: No module named 'quandl'

In [ ]:
# Make sure you adjust the start and end date accordingly
# so that the start date = today date

start = datetime.datetime(2013, 11, 12)
end = datetime.datetime(2018, 11, 12)

amzn = web.DataReader('WIKI/AMZN', 'quandl', start, end)

### Option 2 - Using Yahoo Finance `yfinance` library

In [ ]:
# Question 1
import yfinance as yf
import matplotlib.pyplot as plt
!conda install pandas-datareader
from pandas_datareader import data as pdr

# 1) Using pandas datareader and Yahoo Finance
yf.pdr_override()

#amzn = pdr.get_data_yahoo('AMZN', period = '60d')

In [ ]:
# 2) Using fix_yahoo_finance library only to download AMZN data

#amzn = yf.download('AMZN', period = '1d')

#print(amzn)

In [ ]:
# If we want to download multipe stocks into a single dataframe:

#all_stocks_list = ['AMZN', 'FB']
#all_stocks = yf.download(all_stocks_list, start = start)

In [ ]:
# Question 2
import numpy as np
import pandas as pd

all_stocks_list = ['AMZN', 'META', 'MMM', 'IBM']
all_stocks = yf.download(all_stocks_list, period = '5y') # 60 months is 5 years

print(all_stocks)

In [ ]:
# Question 3

resampled_stocks = all_stocks.resample('BM').last() # Resamples for the business month end
adj_close_stocks = resampled_stocks['Adj Close'] # Only shows the Adj Close value
print(adj_close_stocks)

In [ ]:
# Question 4

print(adj_close_stocks.isnull().values.any()) # No Null values in the dataframe

for ticker in adj_close_stocks:
    plot = pd.plotting.autocorrelation_plot(adj_close_stocks[ticker], label=ticker)

plot.set_title("Autocorrelation Graph of Adjusted Close Month End Stocks")
plot.set_ylabel("Correlation")
plot.set_xlabel("Lag (Months)")

# The autocorrelation plot shows a strong positive correlation between previous stock prices 
# with current prices for the first 2 months. The exception being 3M and IBM which have a moderate correlation.
# After there is a moderate than weak correlation for the remaining months.

In [ ]:
plt.plot(adj_close_stocks)
plt.show()

In [ ]:
# Question 5

# Moves the dates 1 month ahead and deletes the inital month
shift_stocks = adj_close_stocks.shift(1) / adj_close_stocks - 1
shift_stocks = shift_stocks.iloc[1:]


In [ ]:
for ticker in shift_stocks:
    plot = pd.plotting.autocorrelation_plot(shift_stocks[ticker], label=ticker)

plot.set_title("Autocorrelation Graph of Monthly Returns")
plot.set_ylabel("Correlation")
plot.set_xlabel("Lag (Months)")

# There is a weak correlation for returns from these stocks.

In [ ]:
# Question 6
import matplotlib.ticker as ticker

axes = pd.plotting.scatter_matrix(shift_stocks, alpha=0.5, figsize=(16,16), diagonal='hist')

for ax in axes.flatten():
    ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f')) 
plt.show()

# There is a weak correlation between the 4 stock returns. 